In [1]:
import dgl
import numpy as np
import torch

from dgl.data.utils import load_graphs
from ogb.nodeproppred import DglNodePropPredDataset

dataset = DglNodePropPredDataset(name = 'ogbn-mag')

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

def node_level_subsampling(g, list_of_nodes, node_numbers):
    subsample_data = {}
    if len(list_of_nodes) == 0:
        raise Error('list of nodes are empty')
    
    for node_type in list_of_nodes:
        subsample_data[node_type]=g.nodes(node_type)[:node_numbers]
    
    return dgl.node_subgraph(g,subsample_data)
    

def create_mask_from_idx(idx,total_nodes):
    mask_array = np.zeros(total_nodes,dtype=bool)
    mask_array[idx]=True
    return mask_array

def load_mag_data():
    graph = load_graphs('./mag_mp.bin')
    _, label = dataset[0]
    g = graph[0][0]
    print(g)
    features = g.ndata['feat']['paper']
    labels = label['paper']
    mask = torch.BoolTensor(create_mask_from_idx(train_idx['paper'], g.num_nodes('paper')))
   
    return g, features, labels, mask

Using backend: pytorch


In [2]:
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# Define a Heterograph Conv model

import dgl.function as fn

class HeteroRGCNLayer(nn.Module):
    def __init__(self, in_size, out_size, etypes):
        super(HeteroRGCNLayer, self).__init__()
        # W_r for each relation
        self.weight = nn.ModuleDict({
                name : nn.Linear(in_size, out_size) for name in etypes
            })

    def forward(self, G, feat_dict):
        # The input is a dictionary of node features for each type
        funcs = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            # Compute W_r * h
            Wh = self.weight[etype](feat_dict[srctype])
            # Save it in graph for message passing
            G.nodes[srctype].data['Wh_%s' % etype] = Wh
            # Specify per-relation message passing functions: (message_func, reduce_func).
            # Note that the results are saved to the same destination feature 'h', which
            # hints the type wise reducer for aggregation.
            funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
        # Trigger message passing of multiple types.
        # The first argument is the message passing functions for each relation.
        # The second one is the type wise reducer, could be "sum", "max",
        # "min", "mean", "stack"
        G.multi_update_all(funcs, 'sum')
        # return the updated node feature dictionary
        return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}

class HeteroRGCN(nn.Module):
    def __init__(self, G, in_size, hidden_size, out_size):
        super(HeteroRGCN, self).__init__()
        # Use trainable node embeddings as featureless inputs.
        embed_dict = {ntype : nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), in_size))
                      for ntype in G.ntypes}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        # create layers
        self.layer1 = HeteroRGCNLayer(in_size, hidden_size, G.etypes)
        self.layer2 = HeteroRGCNLayer(hidden_size, out_size, G.etypes)

    def forward(self, G):
        h_dict = self.layer1(G, self.embed)
        h_dict = {k : F.leaky_relu(h) for k, h in h_dict.items()}
        h_dict = self.layer2(G, h_dict)
        # get paper logits
        return h_dict['paper']

In [4]:
g, features, labels, mask = load_mag_data()
print(g)

Graph(num_nodes={'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389},
      num_edges={('author', 'ai', 'institution'): 1043998, ('author', 'ap', 'paper'): 7145660, ('field_of_study', 'fp', 'paper'): 7505078, ('institution', 'ia', 'author'): 1043998, ('paper', 'pa', 'author'): 7145660, ('paper', 'pf', 'field_of_study'): 7505078, ('paper', 'pp', 'paper'): 5416271},
      metagraph=[('author', 'institution', 'ai'), ('author', 'paper', 'ap'), ('institution', 'author', 'ia'), ('paper', 'author', 'pa'), ('paper', 'field_of_study', 'pf'), ('paper', 'paper', 'pp'), ('field_of_study', 'paper', 'fp')])
Graph(num_nodes={'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389},
      num_edges={('author', 'ai', 'institution'): 1043998, ('author', 'ap', 'paper'): 7145660, ('field_of_study', 'fp', 'paper'): 7505078, ('institution', 'ia', 'author'): 1043998, ('paper', 'pa', 'author'): 7145660, ('paper', 'pf', 'field_of_study'): 7505078, ('paper'

In [6]:
model = HeteroRGCN(g, 128, 20, 349)
# model = RGCN(len(features), 128, 349, ['cites'])
# paper_feats = features[:10000]
paper_feats = features
author_feats = torch.zeros([g.num_nodes('author'), 128])
fos_feats = torch.zeros([g.num_nodes('field_of_study'), 128])
institute_feats = torch.zeros([g.num_nodes('institution'), 128])
train_mask = mask
print(paper_feats.shape)
print(author_feats.shape)
node_features = {'paper': paper_feats, 'author':author_feats, 'field_of_study':fos_feats, 'institution': institute_feats}
# h_dict = model(g, {'paper': paper_feats})
g.etypes

C:\anaconda\envs\nndlenv\lib\site-packages\torch\nn\modules\container.py:550: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


torch.Size([736389, 128])
torch.Size([1134649, 128])


['ai', 'ap', 'fp', 'ia', 'pa', 'pf', 'pp']

In [8]:
labels

tensor([[246],
        [131],
        [189],
        ...,
        [266],
        [289],
        [  1]])

In [7]:
# Create the model. The output has three logits for three classes.
# model = HeteroRGCN(g, 500, 128, 349)

opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

best_val_acc = 0
best_test_acc = 0

for epoch in range(1):
    model.train()
    # forward propagation by using all nodes and extracting the user embeddings
    logits = model(g)
    loss = F.cross_entropy(logits[train_idx['paper']], labels[train_idx['paper'].squeeze(1)])

    pred = logits.argmax(1)

    opt.zero_grad()
    loss.backward()
    opt.step()



RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:73] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1583970004 bytes. Buy new RAM!

In [ ]:
print(min(loss_array))